Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9


Wrote a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, November 27
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [22]:
import numpy as np
import random
from tqdm import tqdm
from copy import deepcopy

from utils.reproduce_functions import *
from utils.mutation_functions import *
from utils.parent_selection_functions import *
from utils.other_fucntions import *
from utils.island_functions import *

import lab9_lib

In [23]:
PROBLEM_SIZE = 10


MU = 15
LAMBDA = 30
strategy = 'plus' # comma or plus

MUTATION_PROB = 0.2
DYNAMIC_MUTATION_PROB = True
DIVERSITY_THRESHOLD = 20

LENGTH_SOLUTION = 1000
NUMBER_GENERATIONS = 3000
COOLDOWN_TIME = 20

NUM_ISLANDS = 50

In [24]:
# Function to perform swapping between islands
mutate = one_bit_flip
reproduce = two_cuts_crossover
parent_selection = roulette

when_to_swap = fitness_based 
swap_individuals_between_islands = fitness_based_migration

In [25]:
def fitness_proxy(pre_solution, fitness, base, island_ix, pop_history=None):
    this_solution = deepcopy(base)
    start_by = island_ix * (LENGTH_SOLUTION/NUM_ISLANDS)
    start_by = int(start_by)

    for i in range(len(pre_solution)):
        this_solution[start_by + i] = pre_solution[i]

    if pop_history is not None:
        new_ind1_eval = pop_history.get(this_solution.tobytes())
        if new_ind1_eval == None:
            new_ind1_eval = fitness(this_solution)
            pop_history[this_solution.tobytes()] = new_ind1_eval
    else:
        new_ind1_eval = fitness(new_ind1)
    
    return new_ind1_eval
         

In [26]:
def ga(fitness, islands, islands_evals, memoization=False, history=None):
    if memoization:
        pop_history = {}
        for i in range(len(islands_evals)):
            for j in range(len(islands_evals[i])):
                pop_history[islands[i][j].tobytes()] = islands_evals[i][j]

    base = generate_random_individual(length=LENGTH_SOLUTION)
    best_value = -1 
    last_change = 0
    
    for generation in tqdm(range(NUMBER_GENERATIONS)):
        for island_ix, island in enumerate(islands):
            parents = island
            parents_evals = islands_evals[island_ix]
            offsprings = []
            offsprings_evals = []

            while len(offsprings) < LAMBDA:
                
                p1, p2 = parent_selection(parents, parents_evals)
    
                # Reproduce Parents
                off_spring = reproduce(p1, p2)
                
               # Mutate Offspring
                if DYNAMIC_MUTATION_PROB:
                    p_div = get_parents_diversity(p1, p2)
                    new_ind1 = mutate(off_spring, \
                                    mutation_probability=(1 - (min(p_div,LENGTH_SOLUTION/2)/(LENGTH_SOLUTION/2))) )
                else:    
                    new_ind1 = mutate(off_spring, mutation_probability=MUTATION_PROB)

                # Evaluate Offspring
                new_ind1_eval = fitness_proxy(new_ind1, fitness, base, island_ix, pop_history)

                offsprings.append(new_ind1)
                offsprings_evals.append(new_ind1_eval)
                if memoization:
                    pop_history[new_ind1.tobytes()] = new_ind1_eval

            # Create new population (with parents if plus, without if comma)
            all_people = (parents if strategy=='plus' else []) + offsprings
            all_evals = (parents_evals if strategy=='plus' else []) + offsprings_evals
            best_people = np.argsort(all_evals)[::-1]

            if history is not None:
                history[island_ix].append(np.max(all_evals))

            parents = []
            parents_evals = []
            for i in range(MU):
                parents.append(all_people[best_people[i]])
                parents_evals.append(all_evals[best_people[i]])
            islands[island_ix] = parents
            islands_evals[island_ix] = parents_evals
 

        best_eval = max([max(x) for x in history])
        if best_eval > best_value:
            best_value = best_eval
            last_change = generation
        elif generation - last_change > COOLDOWN_TIME:
            break
    
    best_ind = []
    for i in range(NUM_ISLANDS):
        best_island_i = islands[i][ np.argmax(islands_evals[i]) ]
        best_ind.append(best_island_i)
    
    best_ind = np.concatenate(best_ind)
    best_ind_eval = fitness(best_ind)
    return best_ind, best_ind_eval

Without memoization

In [27]:
'''
fitness = lab9_lib.make_problem(PROBLEM_SIZE)
islands = [ [generate_random_individual(length=LENGTH_SOLUTION) for _ in range(50) ] for _ in range(NUM_ISLANDS) ]
islands_evals = [[fitness(x) for x in island] for island in islands ]
parents, parents_evals = ga(fitness, islands, islands_evals, memoization=False)
i_best = np.argmax(parents_evals)
print(parents[i_best])
print("Best score: ", parents_evals[i_best])
print("Num fitness calls: ", fitness.calls)
'''

'\nfitness = lab9_lib.make_problem(PROBLEM_SIZE)\nislands = [ [generate_random_individual(length=LENGTH_SOLUTION) for _ in range(50) ] for _ in range(NUM_ISLANDS) ]\nislands_evals = [[fitness(x) for x in island] for island in islands ]\nparents, parents_evals = ga(fitness, islands, islands_evals, memoization=False)\ni_best = np.argmax(parents_evals)\nprint(parents[i_best])\nprint("Best score: ", parents_evals[i_best])\nprint("Num fitness calls: ", fitness.calls)\n'

With memoization

In [28]:
fitness_calls = []
best_scores = []
for i in range(30):
    fitness = lab9_lib.make_problem(PROBLEM_SIZE)
    
    history = [[] for _ in range(NUM_ISLANDS)]
    islands = [ [generate_random_individual(length=LENGTH_SOLUTION//NUM_ISLANDS) for _ in range(50) ] for _ in range(NUM_ISLANDS) ]
    islands_evals = [[fitness(x) for x in island] for island in islands]
    parents, parents_evals = ga(fitness, islands, islands_evals, memoization=True, history=history)
    print(max([max(x) for x in history]))
    #i_best = np.argmax(parents_evals)
    #print(parents[i_best])
    #print("Best score: ", parents_evals[i_best])
    print("Num fitness calls: ", fitness.calls)
    fitness_calls.append(fitness.calls)
    best_scores.append(np.max(history))

print("average fitness calls: ", np.mean(fitness_calls))
print("average best score: ", np.mean(best_scores))

  0%|          | 0/3000 [00:00<?, ?it/s]

  1%|          | 21/3000 [00:09<21:45,  2.28it/s]


0.8
Num fitness calls:  34919


  1%|          | 21/3000 [00:08<21:05,  2.35it/s]


0.8
Num fitness calls:  35080


...

1.0
Num fitness calls:  35011


  1%|          | 21/3000 [00:08<21:13,  2.34it/s]


0.8
Num fitness calls:  35061


  1%|          | 21/3000 [00:07<18:32,  2.68it/s]

0.8
Num fitness calls:  34284
average fitness calls:  34786.0
average best score:  0.8333333333333335
